# Preamble
Various global variables, parameters


In [ ]:
DEBUG = False
DATASET_MAX_SIZE = 1_000_000

# Fix seed for reproducibility
import numpy as np
np.random.seed(123456)

# Data preparation

This work is using the Million Song Dataset, specifically the one with user listenings count.
It's available at http://millionsongdataset.com/tasteprofile/.
> Thierry Bertin-Mahieux, Daniel P.W. Ellis, Brian Whitman, & Paul Lamere (2011). The Million Song Dataset. In Proceedings of the 12th International Conference on Music Information Retrieval (ISMIR 2011).

It consists of a huge list (48m+ entries) of triplets `(user id, song id, listnings count)`.

## Dataset parsing

Read from the text file, simple parse and convert to numpy data structures.

In [ ]:
# Maps users and songs to their unique index for further referencing as matrix index
USER_MAPPING: dict[str, int] = {}
SONG_MAPPING: dict[str, int] = {}

# It's a list of tuples (user, song, listening count)
dataset_triplet_dtype = np.dtype(
    [
        ("User index", np.uint32),
        ("Song index", np.uint32),
        ("Listening count", np.float64),
    ]
)
dataset_raw: list[tuple[int, int, int]] = []
with open("../train_triplets.txt", "r") as dataset_file:
    for line in dataset_file:
        user_id, song_id, listening_count = line.split("\t")

        dataset_raw.append(
            (
                USER_MAPPING.setdefault(user_id, len(USER_MAPPING)),
                SONG_MAPPING.setdefault(song_id, len(SONG_MAPPING)),
                int(listening_count),
            )
        )

        if len(dataset_raw) >= DATASET_MAX_SIZE:
            break

dataset = np.array(dataset_raw, dtype=dataset_triplet_dtype)

print(dataset.dtype)
print(dataset)
print(
    f"Parsed {len(SONG_MAPPING)} and {len(USER_MAPPING)} users, for a total of {len(dataset)} triplets."
)

## Dataset normalization

Standardize: remove mean and divide by standard deviation.
Hence, values are unitless, centered around 0, and spans in $[-1,1]$.

In [ ]:
dataset["Listening count"] = (
    dataset["Listening count"] - dataset["Listening count"].mean()
) / dataset["Listening count"].std()

print(dataset.dtype, dataset.shape)
print(dataset["Listening count"])

## Dataset training/validation preparation

Shuffle & split into subsets.

We take 2/3 for the training, and 1/3 for validation.

In [ ]:
training_set_size = int(len(dataset) * 0.66)

dataset_perm = np.random.permutation(len(dataset))
dataset_shuffled = dataset[dataset_perm]

train_set = dataset_shuffled[:training_set_size]
validation_set = dataset_shuffled[training_set_size:]

# Learning

The method used is a Stochastic Gradient Descent (SGD), using Regularized Mean Squared Error (RMSE) as the loss function.
It corresponds to
$$
\min_{q^*,p^*} \sum_{(u,i) \in \mathcal{K}} \left(r_{ui} - q_i^Tp_u\right)^2 + \lambda\left(||q_i||^2 + ||p_u||^2\right)
$$

The overall method is taken from [Matrix Factorization Techniques for Recommender Systems
](https://ieeexplore.ieee.org/document/5197422).
> Y. Koren, R. Bell and C. Volinsky, "Matrix Factorization Techniques for Recommender Systems," in Computer, vol. 42, no. 8, pp. 30-37, Aug. 2009, doi: 10.1109/MC.2009.263. keywords: {Recommender systems;Motion pictures;Filtering;Collaboration;Sea measurements;Predictive models;Genomics;Bioinformatics;Nearest neighbor searches;Computational intelligence;Netflix Prize;Matrix factorization},



## Prepare learning

Prepare learning sets $q$ and $p$, which are random matrices of shapes $(|\mathrm{songs}|, l)$ and $(|\mathrm{users}|, l)$.

Set parameters:
- Size `l` of latent space to embed users & films
- Learning rate $\gamma$
- Regularization $\lambda$
- Number of epochs (rounds)

In [ ]:
# (Hyperparameter) Size of latent space to make the embeddings
l = 100
# Initial (random) values
# Shape: (#SONGS, l)
q = np.random.random_sample((len(SONG_MAPPING), l))
if DEBUG:
    print(q.shape, q.dtype, q)
# Shape: (#USERS, l)
p = np.random.random_sample((len(USER_MAPPING), l))
if DEBUG:
    print(p.shape, p.dtype, p)

average_listening_count = train_set["Listening count"].mean()

b_song = np.random.random_sample(len(SONG_MAPPING))
if DEBUG:
    print(b_song.shape, b_song.dtype, b_song)
# Shape: (#USERS, l)
b_user = np.random.random_sample(len(USER_MAPPING))
if DEBUG:
    print(b_user.shape, b_user.dtype, b_user)


# Training parameters
lbd = 0.001
gamma = 0.0005
n_epochs = 100

## Actual learning

Process the SGD, accumulating loss so it can be analyzed.

In [ ]:
from collections.abc import Iterable
from typing import TypedDict, cast


class LearningStats(TypedDict):
    """
    Learning stats (losses: train & validation) for each epoch.
    """

    losses_train: list[np.float64 | float]
    losses_validation: list[np.float64 | float]
    accuracy_train: list[np.float64 | float]
    accuracy_validation: list[np.float64 | float]


def train(l: int, lbd: float, gamma: float, n_epochs: int) -> LearningStats:
    learning_stats: LearningStats = {
        "losses_train": [np.nan] * n_epochs,
        "losses_validation": [np.nan] * n_epochs,
        "accuracy_train": [np.nan] * n_epochs,
        "accuracy_validation": [np.nan] * n_epochs,
    }

    print(f"Training with l={l}, lambda={lbd}, gamma={gamma} for {n_epochs} epochs.")

    for epoch in range(n_epochs):
        print(f"Epoch {epoch+1}")
        loss_sum: float = 0
        accuracy_sum: float = 0

        np.random.shuffle(train_set)  # Reorder each epoch
        # user \in [0, #USERS - 1]
        # song \in [0, #SONGS - 1]
        # listenings \in N (r_ui, "true" value)
        for i, (user, song, listening_count) in enumerate(
            cast(Iterable[tuple[np.uint32, np.uint32, np.float64]], train_set)
        ):
            if DEBUG:
                print(
                    f"Training value {i}/{len(train_set)}: ({user},{song},{listening_count})"
                )

            # Predicted value
            p_u = p[user].copy()
            q_i = q[song].copy()
            b_u = b_user[user].copy()
            b_i = b_song[song].copy()

            listenings_hat = (p_u.T @ q_i) + average_listening_count + b_u + b_i

            # Prediction error
            e_ui = listening_count - listenings_hat

            # This is the learning part
            q[song] += gamma * (e_ui * p_u - lbd * q_i)
            p[user] += gamma * (e_ui * q_i - lbd * p_u)
            b_user[user] += gamma * (e_ui - lbd * b_u)
            b_song[song] += gamma * (e_ui - lbd * b_i)

            # Loss OUTDATED
            loss = e_ui**2 + lbd * (np.linalg.norm(q_i) ** 2 + np.linalg.norm(p_u) ** 2)
            loss_sum += loss

            # Accuracy
            accuracy = e_ui**2
            accuracy_sum += accuracy

        learning_stats["losses_train"][epoch] = loss_sum
        learning_stats["accuracy_train"][epoch] = np.sqrt(accuracy_sum / len(train_set))

        # Now evaluating on validation data
        loss_validation_sum = 0
        accuracy_validation_sum = 0
        for user, song, listening_count in validation_set:
            listenings_hat = p[user].T @ q[song] + average_listening_count + b_user[user] + b_song[song]

            e_ui = listening_count - listenings_hat

            # Loss
            loss = e_ui**2 + lbd * (
                np.linalg.norm(q[song]) ** 2 + np.linalg.norm(p[user]) ** 2
            )
            loss_validation_sum += loss

            # Accuracy
            accuracy = e_ui**2
            accuracy_validation_sum += accuracy

        learning_stats["losses_validation"][epoch] = loss_validation_sum
        learning_stats["accuracy_validation"][epoch] = np.sqrt(
            accuracy_validation_sum / len(validation_set)
        )

        print(
            f"Loss (train): {learning_stats['losses_train'][epoch]}, loss (validation): {learning_stats['losses_validation'][epoch]}"
        )
        print(
            f"Accuracy (train): {learning_stats['accuracy_train'][epoch]}, Accuracy (validation): {learning_stats['accuracy_validation'][epoch]}"
        )

    return learning_stats


training_stats = train(l, lbd, gamma, n_epochs)

# Analysis

## Learning results

We first analyze the learning raw results: training and validation losses.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

# Loss
fig, ax = plt.subplots(figsize=(10, 5))

_ = ax.plot(training_stats["losses_train"], label="Train loss")
_ = ax.plot(training_stats["losses_validation"], label="Validation loss")
_ = ax.set_yscale("log")
_ = ax.set_xlabel("Epoch")
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
_ = ax.set_ylabel("Loss")
_ = ax.set_title("Losses during learning")
_ = fig.legend()

# Accuracy
fig, ax = plt.subplots(figsize=(10, 5))

_ = ax.plot(training_stats["accuracy_train"], label="Train accuracy")
_ = ax.plot(training_stats["accuracy_validation"], label="Validation accuracy")
_ = ax.set_yscale("log")
_ = ax.set_xlabel("Epoch")
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
_ = ax.set_ylabel("Accuracy (RMSE)")
_ = ax.set_title("Accuracy during learning")
_ = fig.legend()

We analyze the differents losses depending on the value of L

In [ ]:
losses_train: list[float] = []
losses_validation: list[float] = []
accuracies_train: list[float] = []
accuracies_validation: list[float] = []
l_values = [20, 30, 40, 50, 60, 70]
n_epochs = 10

training_set_size = int(len(dataset) * 0.66)

dataset_perm = np.random.permutation(len(dataset))
dataset_shuffled = dataset[dataset_perm]

train_set = dataset_shuffled[:training_set_size]
validation_set = dataset_shuffled[training_set_size:]

for l in l_values:
    print(f"Training for l={l}")
    q = np.random.random_sample((len(SONG_MAPPING), l))
    p = np.random.random_sample((len(USER_MAPPING), l))

    LearningStats = train(l, lbd, gamma*10, n_epochs)
    losses_train.append(LearningStats["losses_train"][-1])
    losses_validation.append(LearningStats["losses_validation"][-1])
    accuracies_train.append(LearningStats["accuracy_train"][-1])
    accuracies_validation.append(LearningStats["accuracy_validation"][-1])

In [ ]:
figL, ax = plt.subplots(figsize=(10, 5))
_ = ax.set_title("Losses depending on L")
_ = ax.set_xlabel("L")
_ = ax.set_ylabel("Loss")
_ = ax.plot(l_values, losses_train, label="Train loss")
_ = ax.plot(l_values, losses_validation, label="Validation loss")
_ = figL.legend()
plt.close(figL)

figL

In [ ]:
figA, ax = plt.subplots(figsize=(10, 5))
_ = ax.set_title("Accuracy depending on L")
_ = ax.set_xlabel("L")
_ = ax.set_ylabel("Accuracy")
_ = ax.plot(l_values, accuracies_train, label="Train accuracy")
_ = ax.plot(l_values, accuracies_validation, label="Validation accuracy")
_ = figA.legend()
plt.close(figA)

figA

## ... more analysis

Evaluation of the model?

# Offline metrics
## Background of a few classical metrics
### Precision
relevant retrived instances / all retrieved instances
### Recall
relevant retrived instances / all relevant instances
## Variant used
For our use case, in order to have a metric that works across content based and collaborative filtering.

It is intuitive to use Precision@k and Recall@k.

In our case for example Precision_predict_music(user, k) would be

Precision_predict_music@k = the actual number of top k favorite songs of a user in the k predicted songs / the k favorite songs of a user that the model predict (true positive / predicted positive)

And another example Recall_predict_user(music, k)

Recall_predict_user@k = the actual number of top k lovers of the song in the k predicted users / the k users that like the song the most (true positive / true positive)

In [ ]:
# Assuming that the model matrices are accessible
def user_favorite(user_id, k):
    """Return k indices of favorite songs of a user"""
    user_index = USER_MAPPING[user_id]
    
    # Filter dataset to get only this user's listening records
    user_records = dataset[dataset["User index"] == user_index]
    
    # Sort by listening count in descending order
    sorted_records = np.sort(user_records, order="Listening count")[::-1]
    
    # Return the top k song indices
    return sorted_records["Song index"][:k]

def music_top_fans(song_id, k):
    """Return k indices of biggest fans of a song"""
    song_index = SONG_MAPPING[song_id]
    
    # Filter dataset to get only this song's listening records
    song_records = dataset[dataset["Song index"] == song_index]
    
    # Sort by listening count in descending order
    sorted_records = np.sort(song_records, order="Listening count")[::-1]
    
    # Return the top k user indices (the biggest fans of this song)
    return sorted_records["User index"][:k]
    
def predict_favorite_music(user_id, k):
    user_index = USER_MAPPING[user_id]
    predicted_scores = p[user_index] @ q.T  # Shape: (num_songs,)
    
    # Get indices that would sort scores in descending order
    top_song_indices = np.argsort(predicted_scores)[::-1]
    
    return top_song_indices[:k]

def predict_top_fans(song_id, k):
    song_index = SONG_MAPPING[song_id]
    predicted_scores = p @ q[song_index]  # Shape: (num_users,)
    
    # Get indices that would sort scores in descending order
    top_user_indices = np.argsort(predicted_scores)[::-1]
    
    return top_user_indices[:k]

def precision_recall_predict_music(user_id, k):
    # Get actual favorites (ground truth)
    actual_top_k = set(user_favorite(user_id, k))
    
    # Get predicted favorites
    predicted_top_k = set(predict_favorite_music(user_id, k))
    
    # Calculate precision and recall
    true_positives = len(actual_top_k & predicted_top_k)
    
    precision = true_positives / k if k > 0 else 0
    recall = true_positives / len(actual_top_k) if len(actual_top_k) > 0 else 0
    
    return (precision, recall)

def precision_recall_predict_users(song_id, k):
    # Get actual top fans (ground truth)
    actual_top_k = set(music_top_fans(song_id, k))
    
    # Get predicted top fans
    predicted_top_k = set(predict_top_fans(song_id, k))
    
    # Calculate precision and recall
    true_positives = len(actual_top_k & predicted_top_k)
    
    precision = true_positives / k if k > 0 else 0
    recall = true_positives / len(actual_top_k) if len(actual_top_k) > 0 else 0
    
    return (precision, recall)

def precision_recall_predict_music_all(k):
    total_precision = 0
    total_recall = 0
    num_users = len(USER_MAPPING)
    
    for user_id in USER_MAPPING:
        # Get actual favorites (ground truth)
        actual_top_k = set(user_favorite(user_id, k))
        
        # Get predicted favorites
        predicted_top_k = set(predict_favorite_music(user_id, k))
        
        # Calculate precision and recall
        true_positives = len(actual_top_k & predicted_top_k)
        
        precision = true_positives / k if k > 0 else 0
        recall = true_positives / len(actual_top_k) if len(actual_top_k) > 0 else 0
        
        total_precision += precision
        total_recall += recall
    
    # Return average precision and recall across all users
    avg_precision = total_precision / num_users if num_users > 0 else 0
    avg_recall = total_recall / num_users if num_users > 0 else 0
    
    return (avg_precision, avg_recall)


def precision_recall_predict_users_all(k):
    total_precision = 0
    total_recall = 0
    num_songs = len(SONG_MAPPING)
    
    for song_id in SONG_MAPPING:
        # Get actual top fans (ground truth)
        actual_top_k = set(music_top_fans(song_id, k))
        
        # Get predicted top fans
        predicted_top_k = set(predict_top_fans(song_id, k))
        
        # Calculate precision and recall
        true_positives = len(actual_top_k & predicted_top_k)
        
        precision = true_positives / k if k > 0 else 0
        recall = true_positives / len(actual_top_k) if len(actual_top_k) > 0 else 0
        
        total_precision += precision
        total_recall += recall
    
    # Return average precision and recall across all songs
    avg_precision = total_precision / num_songs if num_songs > 0 else 0
    avg_recall = total_recall / num_songs if num_songs > 0 else 0
    
    return (avg_precision, avg_recall)

## Example usage

In [ ]:
precision_recall_predict_music_all(1)
# return (precision, recall) averaging favorite song of every user

## Important
The example above takes a unreasonable amout of time to finish for dataset size of 1 million. Tested with dataset of 1000 samples.